## Graph construction using networkx and graph-tool

In [1]:
import networkx as nx
import numpy as np
from itertools import product
import numpy.ma as ma
from toolz.curried import pipe, curry
from toolz.curried import map as fmap
import zipfile
import pandas
import time

from graph_tool.all import *

Import data files 

In [4]:
zip_stream = zipfile.ZipFile('../data/cahn-hilliard.zip', 'r')

def read_data(file_name):
    return np.array(
        pandas.read_csv(
            zip_stream.open(file_name, 'r'),
            delimiter=' ',
            header=None               
        ).swapaxes(0, 1)
    )

data = np.array(
    list(
        map(read_data, zip_stream.namelist()[1:6])
    )
)

In [5]:
cond = lambda x: (x[:, 1] != -1) & (x[:, 0] != -1)


def merge_edges(neighbors, ids):
    return pipe(
        ids,
        lambda x: np.reshape(x.flatten(), (-1, 1, 1)),
        lambda x: np.repeat(x, neighbors.shape[1], axis=1),
        lambda x: np.concatenate((x, neighbors), axis=-1).reshape(-1, 2),
        lambda x: x[cond(x)],
    )


def index_vectors(nx, ny, nz):
    index2D = ([1, 0, 0], [1, 1, 0], [0, 1, 0], [-1, 1, 0])
    if ny == nz == 1:
        return ([1, 0, 0],)
    if nz == 1:
        return index2D
    else:
        return index2D + tuple([x, y, 1] for x in (1, 0, -1) for y in (1, 0, -1))


def make_ids_padded(ids):
    nx, ny, nz = ids.shape
    ids_padded = -np.ones((nx + 2, ny + 2, nz + 2), dtype=int)
    ids_padded[1:-1, 1:-1, 1:-1] = ids
    return ids_padded


@curry
def make_sub_ids(ids_padded, indices):
    nx, ny, nz = ids_padded.shape
    left, up, front = indices
    return ids_padded[
        1 + left : nx - 1 + left, 1 + up : ny - 1 + up, 1 + front : nz - 1 + front
    ][..., None]


def make_neighbors(ids):
    nx, ny, nz = ids.shape
    ids_padded = make_ids_padded(ids)

    return pipe(
        index_vectors(nx, ny, nz),
        fmap(make_sub_ids(ids_padded)),
        list,
        lambda x: np.concatenate(x, axis=-1).reshape(nx * ny * nz, len(x), 1),
    )


def make_grid_edges(nx=1, ny=1, nz=1):
    ids = np.arange(nx * ny * nz).reshape(nx, ny, nz)
    return merge_edges(make_neighbors(ids), ids)


def make_grid_graph(shape):
    g = nx.Graph()
    g.add_nodes_from(np.arange(np.prod(shape)))
    g.add_edges_from(make_grid_edges(*shape))
    return g

def make_grid_graph_gt(shape):
    g = Graph(directed = False)
    g.add_vertex(np.prod(shape))
    g.add_edge_list(make_grid_edges(*shape))
    return g


In [6]:
start_time = time.time()
g = make_grid_graph_gt(data.shape)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.489804744720459 seconds ---


In [7]:
start_time = time.time()
g = make_grid_graph(data.shape)
print("--- %s seconds ---" % (time.time() - start_time))

--- 5.758615016937256 seconds ---


In [9]:
make_grid_graph_gt([2, 2])

<Graph object, undirected, with 4 vertices and 6 edges, at 0x1666e6350>

In [9]:
def makeImageGraph(morph):

    G = make_grid_graph(morph.shape)
    vertex_colors = morph.flatten()
    mapping = {(i): vertex_colors[i] for i in range(len(vertex_colors))}
    nx.set_node_attributes(G, mapping, name="color")
    return G


def count_of_vertices(G, phase):
    
    phases = nx.get_node_attributes(G, "color")
    phase_list = list(phases.values())
    return phase_list.count(phase)


def node_phaseA(n, G):
    nodes = G.nodes
    return nodes[n]["color"] == 0


def node_phaseB(n, G):
    nodes = G.nodes
    return nodes[n]["color"] == 1


def makeInterfaceEdges(G):
   
    interface = [
        (n, u)
        for n, u in G.edges()
        if (node_phaseA(n, G) and node_phaseB(u, G))
        or (node_phaseB(n, G) and node_phaseA(u, G))
    ]
    G.remove_edges_from(interface)
    G.add_node(-1, color="green")
    interface = np.unique(np.array(interface))
    interface_edges = [(x, -1) for x in interface]
    G.add_edges_from(interface_edges)
    return G


def makeConnectedComponents(G, phase):

    nodes = (node for node, data in G.nodes(data=True) if data.get("color") == phase)
    subgraph = G.subgraph(nodes)
    subgraph.nodes
    return nx.number_connected_components(subgraph)


def interfaceArea(G):
    
    nodes_0 = [
        neighbor for neighbor in G.neighbors(-1) if G.nodes[neighbor]["color"] == 0
    ]
    nodes_1 = [
        neighbor for neighbor in G.neighbors(-1) if G.nodes[neighbor]["color"] == 1
    ]
    return G.degree[-1], len(nodes_0), len(nodes_1)


def shortest_distances_all(G):
    
    path = nx.single_source_shortest_path(G, -1)
    del path[-1]
    path_length = [len(p) for p in path.values()]
    # print(path_length)
    return sum(path_length) / len(path_length)


def shortest_distances_phase(G, phase):
   
    source = [node for node, data in G.nodes(data=True) if data.get("color") == phase]
    path = [
        nx.shortest_path(G, s, target=-1, weight=None, method="dijkstra")
        for s in source
    ]
    path_length = [len(p) for p in path]
    return sum(path_length) / len(path_length)


def shortest_dist_boundary(G, phase):
    path = nx.single_source_shortest_path(g, -1)
    path_length = [len(p) for p in path.values()]
    return sum(path_length) / len(path_length)


def tortuosity(G, phase):
    return None


def interface_boundary(G, phase):
    return None


def getGraspiDescriptors(data):
   
    

    g = makeImageGraph(data)
    
   # g = makeInterfaceEdges(g)
   # [interface_area, phase_0_interface, phase_1_interface] = interfaceArea(g)

#    return dict(
#        phase_0_count=count_of_vertices(g, 0),
#        phase_1_count=count_of_vertices(g, 1),
#        phase_0_cc=makeConnectedComponents(g, 0),
#        phase_1_cc=makeConnectedComponents(g, 1),
#        interfacial_area=interface_area,
#        phase_0_interface=phase_0_interface,
#        phase_1_interface=phase_1_interface,
#        distance_to_interface=shortest_distances_all(g),
#        distance_to_interface_0=shortest_distances_phase(g, 0),
#        distance_to_interface_1=shortest_distances_phase(g, 1),
#    )


--- 0.01800990104675293 seconds ---


array([[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
   